In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import walk
import pickle
import os

FILEPATH = 'dat'
save = False # export files

In [2]:
batch_files = []
data_files  = []

for (dirpath, dirnames, fnames) in walk(FILEPATH): #[dat, pilot_dat]
    for f in fnames:
        if 'Batch_' in f:
            batch_files.append(f)
        else:
            data_files.append(f)

In [3]:
batch_files, data_files

(['Batch_4318249_batch_results.csv',
  'Batch_4319068_batch_results.csv',
  'Batch_4319396_batch_results.csv',
  'Batch_4322859_batch_results.csv',
  'Batch_4323327_batch_results.csv',
  'Batch_4324179_batch_results.csv'],
 ['test (1).xlsx', 'test.xlsx'])

In [4]:
df_batch = []
df_dat = []
df_comment = []

for f in batch_files:
    df_batch.append(pd.read_csv(FILEPATH + '/' + f))

for f in data_files:
    with pd.ExcelFile(FILEPATH + '/' + f, engine='openpyxl') as xls:
        df_dat.append(pd.read_excel(xls, 'DB'))
        df_comment.append(pd.read_excel(xls, 'COMMENTS'))

# concatenate all data files in a single dataframe
df_dat_whole = pd.DataFrame(columns=df_dat[0].columns)

for dat in df_dat:
    df_dat_whole = df_dat_whole.append(dat)

In [5]:
df_comment[0].columns

Index(['id', 'user', 'comment'], dtype='object')

In [6]:
# compile a user comment dataframe
new_df_comment = pd.DataFrame(columns=df_comment[0].columns)

for df in df_comment:
    new_df_comment = new_df_comment.append(df, ignore_index=True)

In [7]:
df_batch[0].columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Answer.surveycode',
       'Approve', 'Reject'],
      dtype='object')

In [8]:
df_batch[0]['AssignmentId']

0    33FOTY3KEP2YUBTWZND9WSE9XTAC1I
1    36AHBNMV1UT5P9ADB8FTUL71PCPYDS
2    3A0EX8ZRNB5CACCTXCONZVJ1MRDYBL
3    3F1567XTNZMKDH6A8ITXV3L9ZJR9QJ
4    3HOSI13XH252RYFMFCSFH0X07OKDD0
5    3KOPY89HMBJHCSDHWJ8E7P61LXXJ3V
6    3QL2OFSM99Z18I0L4AB7TZRC0U6CNH
7    3S96KQ6I9PL98NBJ12SUM4FMAA7TDY
8    3SB4CE2TJYCIRXHRM22PL6SM23ZAXX
9    3SEPORI8WQG7WSH6NKQAIR8XA3XAZU
Name: AssignmentId, dtype: object

In [9]:
# worker batch file
new_df_batch = pd.DataFrame(columns=['WorkerId', 'WorkTimeInSeconds' , 'Answer.surveycode'])

for i in df_batch:
    temp = i[['WorkerId', 'WorkTimeInSeconds' , 'Answer.surveycode']]
    new_df_batch = new_df_batch.append(temp, ignore_index=True)

old_df_batch = new_df_batch.copy()

new_df_batch

WorkerId WorkTimeInSeconds                     Answer.surveycode
0     A65N7P7LMZ4GE              4587  8701315a-59b9-4fa6-9f5f-4c85ad143a76
1    A2VRDE2FHCBMF8              1474  a2cdf1da-5aa1-40c8-94ed-471a4b1fd78d
2    A34CPKFZXBX1PO              1255  eb0162e1-dd64-4d42-a44a-f7154ae858bc
3     AG9LMLEPXP2YC              4801  36c04674-62c8-4f4c-a1b1-ec6a12badf01
4     AFSW5HVZ0EKET              3046  94d0f9fd-14dd-4191-963e-e4b145625acc
..              ...               ...                                   ...
145  A19ZWBQT8A3LIR              2222  6264fbc4-f71d-4fbf-ad47-8593724dc742
146  A38DXFI1TZA295              5894  520a33dc-5e7f-44f3-85e7-968a80d5ba52
147  A25R2OI9L2Q1OW              5615  7aae45c9-1aa0-415b-a60a-70deda6437f2
148  A3I40B0FATY8VH              2722  189efc73-db8a-428d-bcf8-c20ce9ab883a
149  A1DD23J1WBGQUU              6391  14acf634-faaa-415e-8096-95e474be0f76

[150 rows x 3 columns]

In [10]:
#find any duplicates
new_df_batch[new_df_batch['WorkerId'].duplicated(keep=False)]

WorkerId WorkTimeInSeconds                     Answer.surveycode
12  A1PTH9KTRO06EG              2233  2f05d213-d605-4d99-9e96-7e6fbffe8114
31  A1PTH9KTRO06EG              6742  7a5bd8df-e71c-4c5f-bcb4-75e23efe3a8c

In [11]:
# drop duplicate workers
new_df_batch = new_df_batch.drop_duplicates('WorkerId')

In [12]:
codes = new_df_batch['Answer.surveycode']
new_df_dat = pd.DataFrame(columns=df_dat[0].columns)

for code in codes:
    if len(code) != 36: #screen out wrong survey code submissions       
        print('Submission Error: {}'.format(code))
        # print('{}: {}'.format(set(new_df_batch[new_df_batch['user'] == code]['user'])), code)
    else:
        for dat in df_dat:
            temp = dat.loc[dat['user'] == code]
            if temp.size != 0:
                temp = temp.drop_duplicates('trial') #removing duplicates likely due to server errors (brings n>100 to n=100)
                new_df_dat = new_df_dat.append(temp)
                pass    

Submission Error: AKSJ3C5O3V9RB (worker ID submitted due to technical difficulties, will message code, please do not reject, thank you very much.)


In [13]:
print('There are {} total MTurkers\nThere are {} Worker IDs\nThere are {} matching completion codes\nThere are {} total registration'.format(old_df_batch.shape[0], new_df_batch.shape[0], new_df_dat['user'].unique().size, df_dat_whole['user'].unique().size))

There are 150 total MTurkers
There are 149 Worker IDs
There are 148 matching completion codes
There are 168 total registration


In [14]:
df_dat_whole = df_dat[0].append(df_dat[1])

newlist = set(df_dat_whole['user'])
for lst in newlist:
    print('{}: {}'.format(lst, df_dat_whole[df_dat_whole['user'] == lst]['trial'].size))

7ce48a3c-1cb8-47a2-8845-4d2948ab1f2a: 100
93b2d37f-1482-4ca1-a360-6b247e94b3f3: 101
d1d59f8d-fc03-40cd-8b2c-0118c8a521b1: 100
76e5c11a-d9ba-4e6c-ae2c-9e67d1ee64de: 28
8d1e7411-5206-4168-a1ec-c8786357e52e: 100
95a9dad4-9d13-4030-a9ff-2ef029176a7d: 100
f16190a3-4d9c-48e2-bd70-6023f7ebaa8a: 100
bde987e9-2350-44d9-9d1b-4bda86299194: 100
8c7f4898-2662-420e-9dac-e5745545ea65: 31
422de2ad-6830-4fb6-b855-2d614e455bb1: 100
6847badd-e25e-45fb-9024-606df1e0c615: 9
0264dd9a-93c3-4bb0-b73c-d8a341c2f119: 100
278256a4-4924-45f7-ab71-511f11df802a: 100
7a6349a3-4b11-41aa-a25e-7007c886b68f: 45
c413b1c4-b834-4e2b-afb5-4a7b2c29a818: 100
c8ef9d48-2e60-45ef-b483-100b61edcf80: 100
f2e5901c-b3a9-49fe-83ae-51e3319edc65: 15
54806146-855d-4038-afd0-d0578230fb09: 100
71a3202a-7e3b-4750-a098-3a36633885f9: 100
ecee83c4-9460-4157-be71-47382a0bf0d4: 100
592ca7e2-09e9-423a-996f-2ac59df76923: 100
84b846d3-6a28-4c3b-814f-eaca107e61fd: 100
b4c46b93-ce8b-48d5-8692-78bac6159279: 100
7600cbb5-e672-430c-96f0-dbaee82ce76c: 10

In [15]:
# output final raw database as an excel file (this does not screen out participants who failed to pass the catch trial)
if save:
    new_df_dat.to_excel('MTurk_data.xlsx', index=False)

In [16]:
# making a score profile
lst_id = set(new_df_dat['user'])

df_score = pd.DataFrame(columns=['user', 'score', 'sample', 'comment'])

for userid in lst_id:
    temp = new_df_dat.loc[new_df_dat['user'] == userid]['score']
    max_score = temp.max()
    sample = temp.size
    if new_df_comment.loc[new_df_comment['user'] == userid]['comment'].size != 0:
        temp = new_df_comment.loc[new_df_comment['user'] == userid]['comment'].iloc[0]
        if str(temp) != 'nan':
            comment = temp
        else:
            comment = 'N/A'
    else:
        comment = 'N/A'
    df_score = df_score.append(pd.DataFrame([[userid, max_score, sample, comment]], columns=['user', 'score', 'sample', 'comment']))

In [17]:
if save:
    df_score.to_excel('MTurk_score_profile.xlsx', index=False)

In [18]:
df_score.loc[df_score['sample'] != 100]

user score sample comment
0  d1da6636-c4d9-4391-b802-564fe2734f17     4     99     N/A
0  36c04674-62c8-4f4c-a1b1-ec6a12badf01     4     99     N/A

In [19]:
# disqualified list
df_score.loc[df_score['score'] < 4]

user score sample  \
0  c413b1c4-b834-4e2b-afb5-4a7b2c29a818     2    100   
0  592ca7e2-09e9-423a-996f-2ac59df76923     2    100   
0  b4c46b93-ce8b-48d5-8692-78bac6159279     2    100   
0  f554e046-a38f-4914-a6d1-65d70f3901a1     1    100   
0  e35a6adb-7dfb-455b-8940-b8bda9e847d1     0    100   
0  0f3f6b64-525c-4b7f-8549-982fb7ccfa75     3    100   
0  b889a15a-1b6e-4270-808c-23fa9fd33582     3    100   
0  d77604e1-cb54-47ac-a425-a1fe2e652624     3    100   
0  98f45756-8595-4464-9829-a1a6a4650eb5     3    100   
0  c1605a02-6872-4ea9-ae2f-bf015d97ac3c     3    100   
0  d63d3464-835c-41dd-b34f-30903be6125e     3    100   
0  64661713-f739-46d3-99c1-3aa7b42975c1     3    100   
0  7242568e-27f0-4f8a-b6fd-2bb48df1b113     2    100   
0  62a23695-36e6-4607-bc97-ed665d2c4404     3    100   
0  6b63101e-e6b5-4d0a-83fd-09ba5d9e97e1     3    100   
0  ce002466-ba12-475f-b8fe-2fdc675164aa     0    100   
0  0965127f-3b1c-4e0b-b89c-4253940b9bda     3    100   
0  432867be-3031-47f9-aadc-d468489a4596     1    100   
0  2949248c-b690-4a9e-9f81-03cbc7b5a278     3    100   
0  20732ea8-cba0-4509-b94f-7b8814fbc098     2    100   
0  1ea752f4-be23-4aa6-a039-7e014600b4d7     2    100   
0  aea693a7-3100-4c96-946d-81961be3dfeb     3    100   

                                             comment  
0                                                N/A  
0                                                N/A  
0                                                N/A  
0  This was interesting, I have no idea how I did...  
0                                               good  
0                                               none  
0                                          good task  
0   This was neat and it all worked well, thank you.  
0                                               None  
0  It was pretty hard to know what the patterns w...  
0                                                N/A  
0                                                N/A  
0                                                N/A  
0                               Very enjoyable study  
0                                                N/A  
0                                               good  
0  I hope I did a decent job on your HIT. Thank y...  
0                                 good and best task  
0                                                N/A  
0                                       Interesting.  
0                                          good work  
0                                                N/A

In [20]:
#exporting user comments only
pd.set_option("max_colwidth", 100)

valid_comments = pd.DataFrame(df_score[df_score['comment'] != 'N/A']['comment']).reset_index(drop=True)
if save:
    valid_comments.to_excel('User_comments.xlsx')
valid_comments.head(50)

comment
0                                                                                                  none
1                                                The experiment was very laggy and crashed on me twice.
2                                                      The site/server is very slow loading each page. 
3                                                                               No feedback; thank you.
4                                I wish there was an example of the four small swirls in the pre test. 
5                                                                                           It was fun!
6   I found some of the harder predictions more interesting (such as the tightly formed spirals). I ...
7                                                                                            Interested
8                                                                                              Enjoyed!
9                                                                        I have no additional comments.
10                                                     The task was fab, thank you for the opportunity.
11                        This was interesting, I have no idea how I did. I would be interested in that
12                                                                                                 good
13                                       It was enjoyable...maybe a bonus would make it more enjoyable.
14                                                                                                 none
15                                                                                                 good
16                                                    Thanks for the opportunity, I hope my data helps.
17                                                                                                 none
18                                                                                            good task
19                        I had to do it twice. The application broke about 25% into it the first time.
20                                                                                                 none
21                                                     This was neat and it all worked well, thank you.
22                                               I found the task to be quite engaging to take part in.
23                                                                                                 None
24          Not sure why the training was so different from the main task? Guessing it was by design...
25             This was extremely slow loading. I ended up waiting for way too long between each trial.
26  It was pretty hard to know what the patterns would be beyond the inner space for the ones where ...
27                                                                         Loaded VERY slowly at first.
28                                          thank you for the opportunity to participate in this study.
29                                                                                            No issues
30                                                                                                 none
31                                                                                           Thank you.
32                              I'm very curious to see how I did.  Some of them were very challenging!
33  Some screens were hard to move the slider. The limited information on the blob clusters was diff...
34                                                                                   good and best task
35                                                                                       That was hard!
36                                                    No negative Comments. Everything will be alright.
37                                                                                               Thanks
38                                      

In [21]:
bool_screen = (df_score['score'] >= 4) #* (df_score['sample'] == 100) #catch trial
id_list = df_score.loc[bool_screen]['user'].tolist() #make a ID list of the subjects meeting the criteria
print('There are {} subjects who passed the catch trials'.format(len(id_list)))

There are 126 subjects who passed the catch trials


In [22]:
new_df_dat_real = pd.DataFrame(columns=new_df_dat.columns)

for id_i in id_list:
    new_df_dat_real = new_df_dat_real.append(new_df_dat.loc[new_df_dat['user'] == id_i])

In [23]:
# compute euclidean distance
new_df_dat_real['d'] = '' # adding new column
new_df_dat_real['y'] = '' # separate column for coordinate

for i in range(new_df_dat_real.shape[0]):
    temp_char = new_df_dat_real.iloc[i]['x'][1:-1]
    comma = temp_char.rfind(',')
    x = float(temp_char[:comma])
    y = float(temp_char[comma+1:])
    new_df_dat_real['x'].iloc[i] = x
    new_df_dat_real['y'].iloc[i] = y
    new_df_dat_real['d'].iloc[i] = np.sqrt(x**2 + y**2)

C:\Users\jongm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [24]:
#extra screening
# for user_i in new_df_dat_real[new_df_dat_real['mtype'] == 2][-300:-200]['user'].unique():
#     new_df_dat_real = new_df_dat_real[new_df_dat_real['user'] != user_i]

In [25]:
#extract [est, real, mtype, d]
output = np.array(new_df_dat_real[['est', 'real', 'mtype', 'x', 'd', 'y', 'user']])
output.shape

(12598, 7)

In [26]:
# final output of the cleaned database
filename = 'MTurk_ds.pickle'

if save:
    with open(filename, 'wb') as f:
        pickle.dump(output, f)

In [27]:
#average time stayed on MTurk
new_df_batch.drop_duplicates('WorkerId')['WorkTimeInSeconds'].mean()/60

55.07997762863535

In [28]:
#average time took to complete
list_duration = []

for id_i in id_list:
    temp_df = new_df_dat_real[new_df_dat_real['user'] == id_i]
    t_1 = temp_df[temp_df['trial'] == 1]['date_created'].iloc[0]
    # t_50 = temp_df[temp_df['trial'] == 50]['date_created'].iloc[0]
    t_100 = temp_df[temp_df['trial'] == 100]['date_created'].iloc[0]
    list_duration.append((t_100-t_1).seconds)

np.mean(list_duration) / 60

16.304100529100527

## Update user list with do not repeat feature

In [29]:
filename = 'User_1237902_workers.csv'

df_MT = pd.read_csv(filename)
df_MT = df_MT.astype(object) #must be object otherwise MTurk does not recognize

In [30]:
df_new_MT = df_MT.copy()

workerid = new_df_batch['WorkerId'].tolist()
c = 0

for i in range(df_new_MT.shape[0]):
    
    if df_new_MT['Worker ID'][i] in workerid:
        df_new_MT['UPDATE-Already Participated JOVO'][i] = str(1)
        c += 1
        print('{}: {}'.format(c, df_new_MT['Worker ID'][i]))

1: A10JXOU89D5RXR
2: A12FCLCVIM2CL3
3: A12FTSX85NQ8N9
4: A13WTEQ06V3B6D
5: A153HLAVH5FILS
6: A17K1CHOI773VZ
7: A183WYXN12P2TJ
8: A18G2CLYSTENK
9: A1969Q0R4Y0E3J
10: A19ZWBQT8A3LIR
11: A1AZ97YYFVWFGC
12: A1CPY1HLCFTIL1
13: A1D6V3CKUT95LJ
14: A1DD23J1WBGQUU
15: A1DZMZTXWOM9MR
16: A1F5E941TP0UIZ
17: A1F9KLZGHE9DTA
18: A1GW09ZJF0RXTB
19: A1LA6CIGBNDOH9
20: A1LRJ4U04532TM
21: A1NLJ1L4VCQYV2
22: A1ODA3Q5H6HTDQ
23: A1P2RQ166VS5BT
24: A1P5FJPAEO3RP7
25: A1P6OXEJ86HQRM
26: A1PTH9KTRO06EG
27: A1QKIA8XRNEXIG
28: A1RV2LERVS0A4H
29: A1SH6BOQT9K5VT
30: A1SHLWKA0UH1IS
31: A1SLJKNSNHOJRN
32: A1W7I6FN183I8F
33: A1XVEKS9O73ERE
34: A1YZ0ETOCJO1B2
35: A1ZI4VU6650Q8B
36: A2196WCNDZULFS
37: A22DF0EWILGRLV
38: A22DVMN2Y3XHWA
39: A22T69YW4WUANF
40: A22W4Y4UI182L2
41: A256FHXGSY0E5D
42: A25R2OI9L2Q1OW
43: A26M997VYVK0E6
44: A26NGLGGFTATVN
45: A270HX8LH9LJ8W
46: A2BBJ85QRHJCQO
47: A2CEGS0NVDBCPT
48: A2CWADQFILGY80
49: A2EJ7U3TZAKROG
50: A2FYFCD16Z3PCC
51: A2G8FL7APWI0YS
52: A2GNG7DWHQA6ZZ
53: A2HU2D8889A2FV
54:

In [31]:
if save:
    df_new_MT.to_csv(filename[:-4] + '_updated.csv', index=False)

In [32]:
# df_new_MT[df_new_MT['Worker ID'] == 'AY5WYV3L02AXW']
df_MT_updated = pd.DataFrame(columns=df_MT.columns)

for ids in new_df_batch['WorkerId']:
    df_MT_updated = df_MT_updated.append(df_new_MT[df_new_MT['Worker ID'] == ids])

df_MT_updated.head()

Worker ID                      Link to Individual Worker Page  \
1284   A65N7P7LMZ4GE   https://requester.mturk.com/workers/A65N7P7LMZ4GE   
812   A2VRDE2FHCBMF8  https://requester.mturk.com/workers/A2VRDE2FHCBMF8   
923   A34CPKFZXBX1PO  https://requester.mturk.com/workers/A34CPKFZXBX1PO   
1400   AG9LMLEPXP2YC   https://requester.mturk.com/workers/AG9LMLEPXP2YC   
1396   AFSW5HVZ0EKET   https://requester.mturk.com/workers/AFSW5HVZ0EKET   

     Number of HITs approved or rejected - Lifetime  \
1284                                              1   
812                                               2   
923                                               2   
1400                                              1   
1396                                              1   

     Number of HITs approved - Lifetime Your Lifetime approval rate  \
1284                                  1                     100.00%   
812                                   2                     100.00%   
923                                   2                     100.00%   
1400                                  1                     100.00%   
1396                                  1                     100.00%   

     Number of HITs approved or rejected - Last 30 days  \
1284                                                  1   
812                                                   1   
923                                                   1   
1400                                                  1   
1396                                                  1   

     Number of HITs approved - Last 30 days Your Last 30 days approval rate  \
1284                                      1                         100.00%   
812                                       1                         100.00%   
923                                       1                         100.00%   
1400                                      1                         100.00%   
1396                                      1                         100.00%   

     Number of HITs approved or rejected - Last 7 days  \
1284                                                 1   
812                                                  1   
923                                                  0   
1400                                                 1   
1396                                                 1   

     Number of HITs approved - Last 7 days  ...  \
1284                                     1  ...   
812                                      1  ...   
923                                      0  ...   
1400                                     1  ...   
1396                                     1  ...   

     UPDATE-Completed Social Touch Study CURRENT-Compensation HIT  \
1284                                 NaN                      NaN   
812                                  NaN                      NaN   
923                                  NaN                      NaN   
1400                                 NaN                      NaN   
1396                                 NaN                      NaN   

     UPDATE-Compensation HIT CURRENT-Already Participated JOVO  \
1284                     NaN                               1.0   
812                      NaN                               1.0   
923                      NaN                               1.0   
1400                     NaN                               1.0   
1396                     NaN                               1.0   

     UPDATE-Already Participated JOVO CURRENT-Action Participation  \
1284                                1                          NaN   
812                                 1                          NaN   
923                                 1                          NaN   
1400                                1                          NaN   
1396                                1                          NaN   

     UPDATE-Action Participation CURRENT BlockStatus UPDATE BlockStatus  \
12

In [33]:
# final check to make sure there is no duplicate
df_MT_updated['Worker ID'].unique().size

149